# Demo of Survival Analysis Package

## Step 1: Database Schema Initiation

In [1]:
from survival_analysis.database_preparation.schema import *

2023-12-08 11:42:06,471 - schema.py - <module> - INFO - (Schema Has Been Created) - line: 142


![Database ERD](survival_analysis/docs/ERD.jpg)

## Step 2: Adding Customer Data to the DB

In [2]:
from survival_analysis.database_preparation.sql_interactions\
    import SqlHandler
import pandas as pd

* **Getting Data from a CSV File**

In [3]:
# For Windows users
data = pd.read_csv(r'Raw Data\telco.csv')

# For Mac Users
#data = pd.read_csv(r'Raw Data/telco.csv')
data.head()

,Customer_ID,region,tenure,Age,Marital_Status,Address_ID,income,Education,Retirement,gender,Voice_Included,Internet_Included,Forward_Included,service_category,churn
0,1,Zone 2,13,44,Married,9,64,College degree,No,Male,No,No,Yes,Basic service,Yes
1,2,Zone 3,11,33,Married,7,136,Post-undergraduate degree,No,Male,Yes,No,Yes,Total service,Yes
2,3,Zone 3,68,52,Married,24,116,Did not complete high school,No,Female,No,No,No,Plus service,No
3,4,Zone 2,33,33,Unmarried,12,33,High school degree,No,Female,No,No,No,Basic service,Yes
4,5,Zone 2,23,30,Married,9,30,Did not complete high school,No,Male,No,No,Yes,Plus service,No


* **Populating the 'DimCustomer' table**

In [4]:
Inst = SqlHandler('sa_db', 'DimCustomer')
Inst.insert_many(data) 
Inst.close_cnxn()

2023-12-08 11:42:16,993 - sql_interactions.py - insert_many - INFO - (BEFORE the column intersection: ['customer_id', 'region', 'tenure', 'age', 'marital_status', 'address_id', 'income', 'education', 'retirement', 'gender', 'voice_included', 'internet_included', 'forward_included', 'service_category', 'churn']) - line: 173
2023-12-08 11:42:16,996 - sql_interactions.py - get_table_columns - INFO - (The list of columns: ['Customer_ID', 'Age', 'Tenure', 'Gender', 'Income', 'Marital_Status', 'Address_ID', 'Education', 'Retirement', 'Churn', 'Region', 'Service_Category', 'Voice_Included', 'Internet_Included', 'Forward_Included']) - line: 136
2023-12-08 11:42:16,998 - sql_interactions.py - insert_many - INFO - (AFTER the column intersection: ['customer_id', 'internet_included', 'retirement', 'voice_included', 'income', 'forward_included', 'education', 'tenure', 'age', 'marital_status', 'address_id', 'churn', 'service_category', 'gender', 'region']) - line: 176
2023-12-08 11:42:17,006 - sql_i

## Step 3: Running the Model to Estimate Customer Churn Rate and CLV

In [5]:
from survival_analysis.model_preparation.model_AFT import AFTModelSelector
from survival_analysis.database_preparation.sql_interactions\
    import SqlHandler
from survival_analysis.utils import format_dataframe
import pandas as pd

* **Getting Data From the DB**

In [6]:
#Initiate the connection
Inst=SqlHandler('sa_db', 'DimCustomer')

#Get data in dataframe format
df = Inst.from_sql_to_pandas(chunksize=100, id_value = 'Customer_ID')

#Close the connection
Inst.close_cnxn()

2023-12-08 11:42:21,917 - sql_interactions.py - from_sql_to_pandas - INFO - (The shape of the chunk: (100, 15)) - line: 219
2023-12-08 11:42:21,922 - sql_interactions.py - from_sql_to_pandas - INFO - (The shape of the chunk: (100, 15)) - line: 219
2023-12-08 11:42:21,926 - sql_interactions.py - from_sql_to_pandas - INFO - (The shape of the chunk: (100, 15)) - line: 219
2023-12-08 11:42:21,929 - sql_interactions.py - from_sql_to_pandas - INFO - (The shape of the chunk: (100, 15)) - line: 219
2023-12-08 11:42:21,933 - sql_interactions.py - from_sql_to_pandas - INFO - (The shape of the chunk: (100, 15)) - line: 219
2023-12-08 11:42:21,937 - sql_interactions.py - from_sql_to_pandas - INFO - (The shape of the chunk: (100, 15)) - line: 219
2023-12-08 11:42:21,941 - sql_interactions.py - from_sql_to_pandas - INFO - (The shape of the chunk: (100, 15)) - line: 219
2023-12-08 11:42:21,945 - sql_interactions.py - from_sql_to_pandas - INFO - (The shape of the chunk: (100, 15)) - line: 219
2023-12-

* **Dumifying the Categorical Variables**

In [7]:
df = format_dataframe(df)

* **Running the Model**
1. Chooses the Best Accelerated Failure Time (AFT) model among Exponential, Weibull,
 Log-Normal & Log-Logistic based on AIC. 
2. Computes Churn Rates & CLV For Each Customer for Specified Number of Periods. 
3. Returns the Results in a Dataframe. 

In [8]:
#Running the AFT model for next 12 time periods
duration_column = 'Tenure' 
event_column = 'Churn_Yes'
primary = 'Customer_ID'

aft_model_selector = AFTModelSelector(df, primary,\
                    duration_column, event_column)
aft_model_selector.select_best_model()
aft_model_selector.fit_and_predict(n_time_periods=12)
aft_model_selector.calculate_clv()
aft_model_selector.predictions_df.head()

2023-12-08 11:42:28,426 - model_AFT.py - select_best_model - INFO - (Weibull AIC: 2958.357126946646) - line: 86
2023-12-08 11:42:28,762 - model_AFT.py - select_best_model - INFO - (Exponential AIC: 3052.729101281061) - line: 86
2023-12-08 11:42:29,808 - model_AFT.py - select_best_model - INFO - (LogNormal AIC: 2950.3002131922785) - line: 86
2023-12-08 11:42:30,423 - model_AFT.py - select_best_model - INFO - (LogLogistic AIC: 2952.2582596356688) - line: 86
2023-12-08 11:42:30,425 - model_AFT.py - select_best_model - WARNING - (
Best Model: LogNormal with AIC: 2950.3002131922785) - line: 92
2023-12-08 11:42:30,518 - model_AFT.py - fit_and_predict - INFO - (The AFT model was run successfully.) - line: 138
2023-12-08 11:42:30,604 - model_AFT.py - calculate_clv - INFO - (The CLV predictions were added successfully.) - line: 188


,customer_id,pred_period,churn_rate,CLV
0,1,1,0.00060,1299.220
1,2,1,0.00013,1299.831
2,3,1,0.00000,1300.000
3,4,1,0.00115,1298.505
4,5,1,0.00006,1299.922


* **Populating 'FactPredictions' Table**

In [9]:
#Make sure table is initially empty
Inst=SqlHandler('sa_db', 'FactPredictions') #initiate connection
Inst.truncate_table() #make sure table is initially empty
Inst.close_cnxn() #Close the connection

#Inserting the results to DB 
Inst=SqlHandler('sa_db', 'FactPredictions') #initiate connection
Inst.insert_many(aft_model_selector.predictions_df)
Inst.close_cnxn() #Close the connection

2023-12-08 11:42:48,010 - sql_interactions.py - close_cnxn - INFO - (Committing the changes) - line: 77
INFO:sql_interactions.py:Committing the changes
2023-12-08 11:42:48,014 - sql_interactions.py - close_cnxn - INFO - (The connection has been closed) - line: 79
INFO:sql_interactions.py:The connection has been closed
2023-12-08 11:42:48,021 - sql_interactions.py - insert_many - INFO - (BEFORE the column intersection: ['customer_id', 'pred_period', 'churn_rate', 'clv']) - line: 173
INFO:sql_interactions.py:BEFORE the column intersection: ['customer_id', 'pred_period', 'churn_rate', 'clv']
2023-12-08 11:42:48,024 - sql_interactions.py - get_table_columns - INFO - (The list of columns: ['pred_period', 'customer_ID', 'CLV', 'Churn_Rate']) - line: 136
INFO:sql_interactions.py:The list of columns: ['pred_period', 'customer_ID', 'CLV', 'Churn_Rate']
2023-12-08 11:42:48,026 - sql_interactions.py - insert_many - INFO - (AFTER the column intersection: ['customer_id', 'pred_period', 'churn_rate'

2023-12-08 11:42:48,103 - sql_interactions.py - insert_many - WARNING - (The data is loaded) - line: 196
2023-12-08 11:42:48,105 - sql_interactions.py - close_cnxn - INFO - (Committing the changes) - line: 77
INFO:sql_interactions.py:Committing the changes
2023-12-08 11:42:48,107 - sql_interactions.py - close_cnxn - INFO - (The connection has been closed) - line: 79
INFO:sql_interactions.py:The connection has been closed


# Step 4: Running the API

Run the 'run.py' file to see the usage of the API!